In [40]:
# --- Cell 1: Install Necessary Packages ---
!apt-get install -y nmap
!apt-get install -y amass
!apt-get install -y dnsutils
!apt-get install -y whois
!pip install requests dnspython python-whois httpx tqdm

# Install Subfinder
!wget https://github.com/projectdiscovery/subfinder/releases/download/v2.5.7/subfinder_2.5.7_linux_amd64.zip
!unzip subfinder_2.5.7_linux_amd64.zip
!chmod +x subfinder
!mv subfinder /usr/local/bin/

# Install whatweb
!apt-get install -y whatweb

# Install testssl.sh
!git clone https://github.com/drwetter/testssl.sh.git

# Install dirsearch
!git clone https://github.com/maurosoria/dirsearch.git

print("✅ Setup Completed!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nmap is already the newest version (7.91+dfsg1+really7.80+dfsg1-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package amass
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dnsutils is already the newest version (1:9.18.30-0ubuntu0.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
whois is already the newest version (5.5.13).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
--2025-04-26 11:50:18--  https://github.com/projectdiscovery/subfinder/releases/download/v2.5.7/subfinder_2.5.7_linux_amd64.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to gi

In [41]:
# --- Cell 2: Import Libraries ---
import csv
import requests
import json
import os
import subprocess
import socket
import dns.resolver
import whois
import ssl
from tqdm import tqdm
from datetime import datetime

print("✅ Libraries Imported!")

✅ Libraries Imported!


In [42]:
# --- Cell 3: Upload input.csv ---
from google.colab import files
uploaded = files.upload()

import pandas as pd

df = pd.read_csv('input.csv')
domains = df['domain'].tolist()
print("✅ Domains Loaded:", domains)

Saving input.csv to input (1).csv
✅ Domains Loaded: ['example.com', 'gerrysdnata.com']


In [43]:
# --- Cell 4: Helper Functions ---

def run_subfinder(domain):
    result = subprocess.getoutput(f"subfinder -d {domain} -silent")
    return result.splitlines()

def is_alive(subdomain):
    try:
        r = requests.get(f"http://{subdomain}", timeout=3)
        return r.status_code < 500
    except:
        return False

def get_dns_records(domain):
    records = {}
    try:
        records['A'] = [r.address for r in dns.resolver.resolve(domain, 'A')]
    except: records['A'] = []
    try:
        records['AAAA'] = [r.address for r in dns.resolver.resolve(domain, 'AAAA')]
    except: records['AAAA'] = []
    try:
        records['MX'] = [r.exchange.to_text() for r in dns.resolver.resolve(domain, 'MX')]
    except: records['MX'] = []
    try:
        records['NS'] = [r.to_text() for r in dns.resolver.resolve(domain, 'NS')]
    except: records['NS'] = []
    return records

def get_whois(domain):
    try:
        w = whois.whois(domain)
        return w.domain_name
    except:
        return "WHOIS lookup failed"

def nmap_scan(domain):
    output = subprocess.getoutput(f"nmap -sV -T4 -p- {domain}")
    return output

def detect_tech(domain):
    try:
        output = subprocess.getoutput(f"whatweb {domain}")
        return output
    except:
        return "Tech detection failed"

def ssl_scan(domain):
    output = subprocess.getoutput(f"./testssl.sh/testssl.sh --fast {domain}")
    return output

def header_security(domain):
    try:
        r = requests.get(f"https://{domain}", timeout=5)
        return dict(r.headers)
    except:
        return {}

def sensitive_dirs(domain):
    output = subprocess.getoutput(f"python3 dirsearch/dirsearch.py -u https://{domain} -e php,html,txt --plain-text-report=temp.txt --quiet")
    if os.path.exists('temp.txt'):
        with open('temp.txt') as f:
            paths = f.read().splitlines()
        os.remove('temp.txt')
        return paths
    return []

def simple_risk_score(results):
    score = 0
    if len(results['open_ports']) > 10:
        score += 20
    if 'HSTS' not in results['headers']:
        score += 20
    if 'outdated' in results['tech_stack']:
        score += 30
    if len(results['sensitive_paths']) > 0:
        score += 30
    return min(score, 100)

print("✅ Helper Functions Ready!")

✅ Helper Functions Ready!


In [44]:
# --- Cell 5: Main Scanner ---

os.makedirs("outputs", exist_ok=True)

for domain in tqdm(domains):
    result = {}
    result['domain'] = domain
    result['scan_date'] = str(datetime.now().date())

    result['subdomains'] = run_subfinder(domain)
    result['live_subdomains'] = [s for s in result['subdomains'] if is_alive(s)]
    result['dns_records'] = get_dns_records(domain)
    result['whois_info'] = get_whois(domain)
    result['open_ports'] = nmap_scan(domain)
    result['tech_stack'] = detect_tech(domain)
    result['ssl_info'] = ssl_scan(domain)
    result['headers'] = header_security(domain)
    result['sensitive_paths'] = sensitive_dirs(domain)

    result['risk_score'] = simple_risk_score(result)
    result['risk_summary'] = "Auto-generated risk summary based on findings."

    with open(f"outputs/{domain}.json", "w") as f:
        json.dump(result, f, indent=2)

print("✅ Scanning Complete! Check 'outputs' folder.")

100%|██████████| 2/2 [13:15<00:00, 397.73s/it]

✅ Scanning Complete! Check 'outputs' folder.
